In [1]:
import pandas as pd
import random
import names
import datetime as dt
import sqlite3

In [2]:
# Converts a 4 digit number into a 5 digit checksum
def checkSum(num):
    numString = str(num)
    checkNum = str(3 * (int(numString[0]) + int(numString[2])) + 7 * (int(numString[1]) + int(numString[3])))[-1]
    numString += checkNum
    return int(numString)

# Generates a random date between two dates
def randomDateString():
    minDate = dt.date(2015,1,1)
    maxDate = dt.date(2022,10,1)
    dateRange = (maxDate-minDate).days
    randDate = minDate + dt.timedelta(days=random.randrange(dateRange))
    return randDate.strftime("%Y-%m-%d")

# Adds a random amount of days to a date
def randomDayAdder(maxYear, yearIn):
    date = dt.datetime.strptime(yearIn,"%Y-%m-%d")
    if date.year < maxYear:
        return (date + dt.timedelta(days=random.randint(4,20))).strftime("%Y-%m-%d")
    else:
        return None
    
# List of genres
genreList = ["Horror","Comedy","Thriller","Fantasy","Non-Fiction","Cooking","History"]

# List of residences
residences = ["Como","Subiaco","Fremantle","Rockingham","Midland","Maylands","Joondalup","Kingsley"]

In [3]:
# Create sample data for BookEdition
isbn = [checkSum(n) for n in random.sample(range(1000,10000), 100)]
author = [names.get_full_name() for n in range(100)]
publicationDate = [random.randint(1900,2022) for n in range(100)]
genre = [genreList[random.randint(0,len(genreList)-1)] for n in range(100)]

BookEdition = pd.DataFrame(list(zip(isbn,author,publicationDate,genre)),
                          columns=["ISBN","author","publicationDate","genre"])

In [4]:
BookEdition.head()

,ISBN,author,publicationDate,genre
0,41298,Stacey Valley,2014,History
1,52762,Angela Hooker,1909,Non-Fiction
2,91413,Dawn Harris,1931,Cooking
3,70843,Martha Reyes,1930,Horror
4,15682,Jesus Perras,1995,History


In [5]:
# Create sample data for BookCopy
isbn_copy = [isbn[random.randint(0,len(isbn)-1)] for n in range(200)]
copyNumber = [isbn_copy[0:i+1].count(item) for i, item in enumerate(isbn_copy)]
daysLoaned = [0] * len(isbn_copy)

BookCopy = pd.DataFrame(list(zip(isbn_copy,copyNumber,daysLoaned)),
                       columns=["ISBN","copyNumber","daysLoaned"])

In [6]:
BookCopy.head()

,ISBN,copyNumber,daysLoaned
0,86563,1,0
1,71163,1,0
2,86783,1,0
3,61139,1,0
4,48118,1,0


In [7]:
# Create sample data for Client
clientId = [i+1 for i in range(60)]
name = [names.get_full_name() for n in range(60)]
residence = [residences[random.randint(0,len(residences)-1)] for n in range(60)]

Client = pd.DataFrame(list(zip(clientId,name,residence)),
                     columns=["clientId","name","residence"])

In [8]:
Client.head()

,clientId,name,residence
0,1,Roxann Vanegas,Como
1,2,Nathaniel Heisler,Maylands
2,3,George Lopez,Como
3,4,Pam Dickinson,Como
4,5,Estella Bushnell,Maylands


In [9]:
# List of tuples ensures ISBN and copyNumber are consistent and not mismatched during random selection
isbn_copyNumber = [list(zip(isbn_copy,copyNumber))[random.randint(0,len(isbn_copy)-1)] for n in range(500)]

clientId_loan = [clientId[random.randint(0,59)] for n in range(500)]
isbn_loan = [item[0] for item in isbn_copyNumber]
copyNumber_loan = [item[1] for item in isbn_copyNumber]
dateOut = [randomDateString() for n in range(500)]
dateBack = ["NULL" for n in range(500)]

loan = pd.DataFrame(list(zip(clientId_loan,isbn_loan,copyNumber_loan,dateOut,dateBack)),
                   columns=["clientId","ISBN","copyNumber","dateOut","dateBack"])

In [10]:
loan.head()

,clientId,ISBN,copyNumber,dateOut,dateBack
0,43,63511,1,2021-11-20,NULL
1,40,88505,4,2018-10-06,NULL
2,26,88505,2,2017-09-02,NULL
3,9,16751,3,2019-09-20,NULL
4,38,34980,1,2020-03-02,NULL


In [11]:
# Connect to the sqlite database
db = input("Enter database name here including .db extension (i.e., myDatabase.db): ")
conn = sqlite3.connect(db)
cursor = conn.cursor()

Enter database name here including .db extension (i.e., myDatabase.db): test.db


In [12]:
# Populate BookEdition table with sample data
BookEdition.to_sql(name="BookEdition", con=conn, if_exists="append", index=False)

100

In [13]:
#Populate BookCopy table with sample data
BookCopy.to_sql(name="BookCopy", con=conn, if_exists="append", index=False)

200

In [14]:
# Populate Client table with sample data
Client.to_sql(name="Client", con=conn, if_exists="append", index=False)

60

In [15]:
# loan.to_sql(name="loan", con=conn, if_exists="append", index=False)

In [16]:
# Commit to adding sample data
conn.commit()

In [17]:
# Generate a list of random return dates for some loans
newDateBack = [randomDayAdder(2022, date) for date in dateOut]

# Row wise simultaneous insertion and update of loan table, see note () in readme for the reason this is done
for i, items in enumerate(loan.itertuples()):
    insert = f"INSERT INTO loan ('clientId','ISBN','copyNumber','dateOut','dateBack') VALUES ({items[1]},'{items[2]}',{items[3]},'{items[4]}',{items[5]});"
    cursor.execute(insert)
    conn.commit()
    if newDateBack[i]:
        update = f"UPDATE loan SET dateBack = '{newDateBack[i]}' WHERE clientId = {items[1]} AND ISBN = '{items[2]}' AND copyNumber = {items[3]} AND dateOut < '{items[5]}' AND dateBack IS NULL"
        cursor.execute(update)
        conn.commit()